In [185]:
import pandas as pd
from pathlib import Path

In [186]:
airafts_data = pd.read_csv("../data/aircrafts.csv")
airafts_data.head()

,aircraft_code,model,count
0,733,Боинг 737-300,130
1,773,Боинг 777-300,402
2,321,Аэробус A321-200,170
3,319,Аэробус A319-100,116
4,CR2,Бомбардье CRJ-200,50


In [187]:
Path.mkdir(Path("../data/results"), exist_ok=True)
airafts_data.to_csv("../data/results/aircrafts.csv", index=False)

In [188]:
import datetime
import rstr

datetime.datetime.today()

rstr.xeger(r"^[A-Z]-[A-Z]{4}|[A-Z]{2}-[A-Z]{3}|N[0-9]{3}[A-Z]{3}$")
type(r"^[A-Z]-[A-Z]{4}|[A-Z]{2}-[A-Z]{3}|N[0-9]{3}[A-Z]{3}$")
datetime.datetime.today()

datetime.datetime(2024, 11, 3, 16, 7, 31, 495094)

In [261]:
import random
from zoneinfo import ZoneInfo


def __generate_dates_for_flight_num(
    min_year: int,
    max_year: int = 2015,
    timezone: str = "Asia/Yekaterinburg",
) -> list[tuple[datetime.datetime, datetime.datetime]]:

    res = []
    count = 0
    step = max(3, (max_year - min_year) // 3)
    for i in range(min_year, max_year, step):
        year = random.randint(i, i + step)
        month = random.randint(1, 12)
        day = random.randint(1, 28)
        date = datetime.datetime(
            year,
            month,
            day,
            hour=0,
            minute=0,
            second=0,
            tzinfo=ZoneInfo(timezone),
        )
        if count > 0:
            res.append(date - datetime.timedelta(days=1))
        res.append(date)
        count += 1
    res.append(
        datetime.datetime(
            year=9999,
            month=12,
            day=31,
            hour=23,
            minute=59,
            second=59,
            tzinfo=ZoneInfo(timezone),
        )
    )
    return res

In [258]:
from bson import ObjectId


def generate_aircraft_number_story(
    aircraft_id: ObjectId,
    pattern: str,
    min_year: int,
    max_year: int = 2015,
    timezone: str = "Asia/Yekaterinburg",
) -> pd.DataFrame:

    dates = __generate_dates_for_flight_num(min_year, max_year, timezone)

    temp_dic = {}
    temp_dic["aircraft_id"] = []
    temp_dic["aircraft_num"] = []
    temp_dic["registration_date"] = []
    temp_dic["deregistration_date"] = []
    for i in range(1, len(dates), 2):

        temp_dic["aircraft_id"].append(str(aircraft_id))
        temp_dic["aircraft_num"].append(
            rstr.xeger(string_or_regex=pattern)
        )
        temp_dic["registration_date"].append(dates[i - 1])
        temp_dic["deregistration_date"].append([dates[i]])

    return pd.DataFrame(temp_dic)

In [265]:
story = generate_aircraft_number_story(
    123,
    pattern=r"^[A-Z]-[A-Z]{4}|[A-Z]{2}-[A-Z]{3}|N[0-9]{3}[A-Z]{3}$",
    min_year=2000,
    max_year=2015,
)
story

,aircraft_id,aircraft_num,registration_date,deregistration_date
0,123,N643SHC,2000-02-18 00:00:00+05:00,[2009-11-12 00:00:00+05:00]
1,123,N826ULM,2009-11-13 00:00:00+05:00,[2011-10-06 00:00:00+06:00]
2,123,N936MQO,2011-10-07 00:00:00+06:00,[9999-12-31 23:59:59+05:00]
